In [1]:
import tensorflow as tf
import numpy
import os
import json, pickle
import pandas
from functools import partial, reduce
import importlib

import sys
sys.path.append('../libs')

import flacdb
import prepare_data
import initialize
import data_pipeline
import loss_metrics
import conv_model
import plot_batch

In [2]:
importlib.reload(initialize)
sig_data, metadata = initialize.load_initial_data()

In [3]:
save_path = '/scr1/mimic/initial_data/'
sig_data.to_pickle(save_path + 'sig_data.pkl')
metadata.to_pickle(save_path + 'metadata.pkl')

In [4]:
(metadata['chunk_count'] == 0).sum()

0

In [2]:
H = initialize.load_hypes()
parts = ['train', 'validation']
tensors, metadata, priors = initialize.run(H, parts=parts)

In [3]:
(metadata['chunk_count'] == 0).sum()

0

In [4]:
counts = tensors['train']['chunk_count']
counts = numpy.array(counts.flat_values)
counts.min()

1

In [3]:
dataset = {part: data_pipeline.build(H, tensors[part], part) for part in parts}
model = conv_model.build(H, priors)

In [4]:
model.fit(
    dataset['train'],
    validation_data = dataset['validation'],
    epochs = 2**20,
    steps_per_epoch = 2**H['steps_per_epoch_log2'],
    validation_steps = 2**H['validation_steps_log2'],
    validation_freq = range(0, 2**20, 2**H['validation_frequency_log2']),
    verbose = 1,
)

Train for 128 steps, validate for 128 steps
Epoch 1/1048576
128/128 [==============================] - 579s 5s/step - loss: 1.0490 - 25000_diabetes_sensitivity: 0.4041 - 25000_diabetes_specificity: 0.5903 - 25000_diabetes_accuracy: 0.4972 - 2720_hypercholesterolemia_sensitivity: 0.2787 - 2720_hypercholesterolemia_specificity: 0.7435 - 2720_hypercholesterolemia_accuracy: 0.5111 - 2724_hyperlipidemia_sensitivity: 0.5649 - 2724_hyperlipidemia_specificity: 0.5089 - 2724_hyperlipidemia_accuracy: 0.5369 - 2761_hyposomality_sensitivity: 0.2824 - 2761_hyposomality_specificity: 0.7051 - 2761_hyposomality_accuracy: 0.4938 - 2762_acidosis_sensitivity: 0.2956 - 2762_acidosis_specificity: 0.6915 - 2762_acidosis_accuracy: 0.4936 - 27651_dehydration_sensitivity: 0.0358 - 27651_dehydration_specificity: 0.8995 - 27651_dehydration_accuracy: 0.4677 - 27652_hypovolemia_sensitivity: 0.0312 - 27652_hypovolemia_specificity: 0.9297 - 27652_hypovolemia_accuracy: 0.4805 - 27800_obesity_sensitivity: 0.1487 - 278

KeyboardInterrupt: 